In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Internazionale                                                           \
                Date   Time          Comp        Round  Day Venue Result   GF   
0         2023-08-19  20:45       Serie A  Matchweek 1  Sat  Home      W  2.0   
1         2023-08-28  20:45       Serie A  Matchweek 2  Mon  Away      W  2.0   
2         2023-09-03  18:30       Serie A  Matchweek 3  Sun  Home      W  4.0   
3         2023-09-16  18:00       Serie A  Matchweek 4  Sat  Home      W  5.0   
4         2023-09-20  21:00  Champions Lg  Group stage  Wed  Away      D  1.0   

                          ... Standard               Expected               \
    GA          Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  0.0             Monza  ...     17.2  1.0  0     0      2.7  2.7    0.12   
1  0.0          Cagliari  ...     15.9  0.0  0     0      0.9  0.9    0.05   
2  0.0        Fiorentina  ...     14.5  1.0  1     1      3.7  2.9    0.15   
3  1.0             Milan  ...     15.2  1.0  1     1      2.6  1.8    0.14   
4  1.0  es Real Sociedad  ...     14.7  0.0  0     0      0.5  0.5    0.09   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -0.7    -0.7        Match Report  
1  1.1     1.1        Match Report  
2  0.3     0.1        Match Report  
3  2.4     2.2        Match Report  
4  0.5     0.5        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,...,3-5-2,Andrea Colombo,Match Report,NaN,22,3,17.2,1.0,0,0
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,...,3-5-2,Michael Fabbri,Match Report,NaN,17,3,15.9,0.0,0,0
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,...,3-5-2,Matteo Marchetti,Match Report,NaN,20,10,14.5,1.0,1,1
3,2023-09-16,18:00,Serie A,Matchweek 4,Sat,Home,W,5.0,1.0,Milan,...,3-5-2,Simone Sozza,Match Report,NaN,13,6,15.2,1.0,1,1
4,2023-09-20,21:00,Champions Lg,Group stage,Wed,Away,D,1.0,1.0,es Real Sociedad,...,3-5-2,Michael Oliver,Match Report,NaN,6,1,14.7,0.0,0,0


In [16]:
years = list(range(2022, 2020, -1))
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [18]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Serie A"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

<ipython-input-18-12c93c094b7f>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = year
<ipython-input-18-12c93c094b7f>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name


In [19]:
len(all_matches)

40

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,...,Match Report,NaN,22.0,3.0,17.2,1.0,0,0,2022,Internazionale
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,...,Match Report,NaN,17.0,3.0,15.9,0.0,0,0,2022,Internazionale
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,...,Match Report,NaN,20.0,10.0,14.5,1.0,1,1,2022,Internazionale
3,2023-09-16,18:00,Serie A,Matchweek 4,Sat,Home,W,5.0,1.0,Milan,...,Match Report,NaN,13.0,6.0,15.2,1.0,1,1,2022,Internazionale
5,2023-09-24,12:30,Serie A,Matchweek 5,Sun,Away,W,1.0,0.0,Empoli,...,Match Report,NaN,23.0,5.0,16.8,0.0,0,0,2022,Internazionale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2023-05-08,18:30,Serie A,Matchweek 34,Mon,Away,L,0,2,Udinese,...,Match Report,NaN,11.0,3.0,21.1,1.0,0,0,2021,Sampdoria
37,2023-05-15,20:45,Serie A,Matchweek 35,Mon,Home,D,1,1,Empoli,...,Match Report,NaN,13.0,3.0,16.5,1.0,0,0,2021,Sampdoria
38,2023-05-20,20:45,Serie A,Matchweek 36,Sat,Away,L,1,5,Milan,...,Match Report,NaN,6.0,3.0,17.5,0.0,0,0,2021,Sampdoria
39,2023-05-26,20:45,Serie A,Matchweek 37,Fri,Home,D,2,2,Sassuolo,...,Match Report,NaN,17.0,3.0,15.0,0.0,0,0,2021,Sampdoria


In [23]:
match_df.to_csv("matches.csv")